# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1602468690205_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://fall2020-cse6242"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [3]:
def user():
    # Returns a string consisting of your GT username.
    return 'yqi82'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips longer than 2 miles.

In [46]:
def long_trips(trips):
    #clean data
    trips = trips.withColumn("passenger_count", col("passenger_count").cast("long")) \
    .withColumn("total_amount", col("total_amount").cast("double")) \
    .withColumn("trip_distance", col("trip_distance").cast("double"))
    # Returns a Dataframe with Schema the same as :trips:
    trips_filtered = trips.filter(trips.trip_distance >= 2.0)
#     trips.show()
    return trips_filtered
#     pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [47]:
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
    trips.show()
    lookup.show()
    trips.withColumn("passenger_count", col("passenger_count").cast("long"))
    trips1 = trips.withColumnRenamed("DOLocationID", "LocationID") \
    .groupBy("LocationID").agg({"passenger_count":"sum"}) \
    .withColumnRenamed("SUM(passenger_count)", "pcount")
    trips2 = lookup.filter(lookup.Borough == "Manhattan")
#     trips2.show()
    mtrips = trips2.join(trips1, on=["LocationID"], how="inner").sort(col("pcount").desc()) \
    .withColumnRenamed("LocationID", "DOLocationID").limit(20)
#     trips3.show()
#     mtrips = trips3.groupBy("LocationID").agg({"passenger_count":"sum"}) \
#     .withColumnRenamed("SUM(passenger_count)", "pcount") \
#     .sort(col("pcount").desc()).withColumnRenamed("LocationID", "DOLocationID").limit(20)
    mtrips.show()
    return mtrips
#     pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [50]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    trips = trips.withColumn("total_amount", col("total_amount").cast("double"))
    trips1 = trips.groupBy("PULocationID").agg({"total_amount":"avg", "PULocationID":"count"}) \
    .withColumnRenamed("AVG(total_amount)", "avg_total_amount").withColumnRenamed("COUNT(PULocationID)", "trip_counts") \
    .withColumn("trip_counts", col("trip_counts").cast("long"))
#     trips1.show()
    trips2 = trips.join(mtrips, on=["DOLocationID"], how="inner")
#     trips2.show()
    trips3 = trips2.groupBy("PULocationID").agg({"PULocationID":"count"}) \
    .withColumnRenamed("COUNT(PULocationID)", "popular_trip_counts") \
    .withColumn("popular_trip_counts", col("popular_trip_counts").cast("long"))
    
#     trips3.show()
    trips4 = trips1.join(trips3, on=["PULocationID"], how="left").na.fill(0)
#     trips4.show()
    trips5 = trips4.withColumn("weighted_profit", (col("avg_total_amount") * col("popular_trip_counts") / col("trip_counts")).cast("double"))
#     trips5.show()
#     trips6 = trips5.withColumn("weighted_profit", col("proportion") * col("avg_total_amount"))
#     trips6.show()
    wp = trips5.select("PULocationID", "weighted_profit").sort(col("weighted_profit").desc())
    wp.show()
    return wp
#     pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [56]:
def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    calc = calc.withColumn("weighted_profit", col("weighted_profit").cast("double"))
    calc1 = calc.withColumnRenamed("PULocationID", "LocationID")
    calc2 = calc1.join(lookup, on=["LocationID"], how="left").na.fill(0.0) \
    .sort(col("weighted_profit").desc())
#     calc2.show()
    final = calc2.select("Zone", "Borough", "weighted_profit").limit(20)
#     final.show()
    return final
#     pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the address to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear ina folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [58]:
bucket = 's3://cse6242-yqi82/output-large1'
# main('small',bucket)
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

yqi82
Trip Count:  187203269
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2018-01-01 00:44:55|  2018-01-01 01:03:05|              1|          2.7|         1|                 N|         239|         140|           2|         14|  0.5|    0.5|         0|           0|                  0.3|        15.3|
|       1| 2018-01-

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>